### Stocks Analysis

In [155]:
#Packages
import pandas as pd
import numpy as np
from selenium import webdriver
from datetime import date
import time
import io
import re
from datetime import datetime

In [85]:
#This function generates hyperlinks to stooq.pl website. Ticker, first and last observation date and number of pages need to be provided
#If number of pages won't be provided 20 hyperlinks will be created by default
#(Take into consideration that if more historical data need to be downloaded default no of pages might not be sufficient)
def generateHyperlinks(ticker,startDate = "2020-01-01",endDate = str(date.today()),no_of_pages = 20):
    startDate = startDate.replace("-","")
    endDate = endDate.replace("-","")
    webpage = r"https://stooq.pl/q/d/?s=" + ticker + '&c=0&d1=' + startDate + "&d2=" + endDate + "&l="
    hyperlinks = [webpage + str(i) for i in range(1,no_of_pages+1)]
    return hyperlinks

In [86]:
#Remember that Stooq.pl has a limit of daily page refreshes
driver = webdriver.Chrome(executable_path=r"C:\Users\woote\Desktop\Python\Projects\chromedriver.exe")
hyperlinks = generateHyperlinks("MVP","2021-01-01",no_of_pages=8)
tableValues = []

for i,link in enumerate(hyperlinks):

    driver.get(link)

    if i == 0:
        try:
            accept = driver.find_element_by_xpath("//p[contains(@class, 'fc-button-label')]")
            accept.click()
        except:
            pass
        time.sleep(3)
    
    try:
        tableText = driver.find_element_by_id("fth1")
        tableValues.append(tableText.text)
        #tableValues["Table {0}".format(i)] = tableText.text
    except:
        break

In [202]:
def dataCleaner(tableValues, colnames = ['id','day','month','year','open','max','min','close','perc_change','abs_change','volume']):
    
    #Dictionary which will help to convert month names to a digits in date column
    months = {"sty": "01","lut": "02","mar": "03","kwi": "04","maj": "05","cze": "06",\
              "lip": "07","sie": "08","wrz": "09","paż": "10","lis": "11","gru": "12"}
    #Joining list elements with new line operator, then deliminating by spaces
    #Dropping the duplicated rows (as we are scraping column headers as well)
    #And erasing first row wiith wrongly assigned headers
    #Deleting rows where 'close' is nan (usually its additional row pointing the date of dividend)
    data = pd.read_csv(io.StringIO('\n'.join(tableValues)), delim_whitespace=True,names=colnames).drop_duplicates().iloc[1:,:]
    data = data[data['close'].notna()]
    data = data.iloc[:-1 , :]
    #Reseting the index (after deleting duplicated rows)
    data.reset_index(inplace = True)
    #Inserting date column
    data.insert(4,'date',pd.to_datetime(data["day"] + "/" + data["month"].map(months) + "/" + data["year"], format='%d/%m/%Y'))
    #Dropping needless columns
    data.drop(['id','day','month','year'],axis = 1,inplace = True)
    #Adjusting data types
    data[['max','min','open','close']] = data[['max','min','open','close']].astype(float)
    data['volume'] = data['volume'].str.replace(",","").fillna(0).astype(int)
    data['perc_change'] = data['perc_change'].str.replace("%","").str.replace("+","").astype(float)
    
    return data

In [203]:
df = dataCleaner(tableValues)

<ipython-input-202-c94af9019e7a>:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data['perc_change'] = data['perc_change'].str.replace("%","").str.replace("+","").astype(float)


In [138]:
#pd.set_option('display.max_rows', None)


In [204]:
df.dtypes

index                   int64
date           datetime64[ns]
open                  float64
max                   float64
min                   float64
close                 float64
perc_change           float64
abs_change            float64
volume                  int32
dtype: object